In [621]:
V = ['S','A','B','C','D','E','F','G','H','I','J','T']
E = [('S','A',10),('S','D',5),('S','C',2),('S','E',4),('A','B',4),('A','G',2),('A','C',2),('C','G',8),('C','E',5),('E','G',7),('E','F',2),('D','J',2),('D','E',3),('J','F',1),('J','I',5),('F','T', 6),('F','I',10),('G','F',2),('G','H',2),('B','H',3),('I','T',7),('H','T',9),('T','G',6)]

In [622]:
for i in V:
    connectionCount = 0
    for tu in E:
        if tu[0] == i or tu[1] == i:
            connectionCount += 1
    print("Vertex:",i,"Connections:",connectionCount)

Vertex: S Connections: 4
Vertex: A Connections: 4
Vertex: B Connections: 2
Vertex: C Connections: 4
Vertex: D Connections: 3
Vertex: E Connections: 5
Vertex: F Connections: 5
Vertex: G Connections: 6
Vertex: H Connections: 3
Vertex: I Connections: 3
Vertex: J Connections: 3
Vertex: T Connections: 4


In [623]:
c = {}
for i in V:
    x1 = {}
    for tu in E:
        if i == tu[0]:
            x1[tu[1]] = tu[2]
    if x1 != {}:
        c[i] = x1


In [624]:
c

{'S': {'A': 10, 'D': 5, 'C': 2, 'E': 4},
 'A': {'B': 4, 'G': 2, 'C': 2},
 'B': {'H': 3},
 'C': {'G': 8, 'E': 5},
 'D': {'J': 2, 'E': 3},
 'E': {'G': 7, 'F': 2},
 'F': {'T': 6, 'I': 10},
 'G': {'F': 2, 'H': 2},
 'H': {'T': 9},
 'I': {'T': 7},
 'J': {'F': 1, 'I': 5},
 'T': {'G': 6}}

In [625]:
from gurobipy import *

In [626]:
m = Model()
x = {}
for i in c:
    x1 = {}
    for j in c[i]:
        varname = str('var'+i+j)
        x1[j] = m.addVar(lb = 0, name=varname)
    x[i] = x1
v = m.addVar(lb = 0, name="maxFlow")


In [627]:
for i in x:
    sum_= 0
    sum = 0
    for j in x[i]:
        sum += x[i][j]
        m.addConstr(x[i][j] <= c[i][j])
    for j in x:
        if i in x[j]:
            sum_ += x[j][i]
    if(i == 'S'):
            m.addConstr(sum - sum_ == v) 
    elif(i == 'T'):
            m.addConstr(sum - sum_ == -v)
    else:
        m.addConstr(sum - sum_ == 0)


In [628]:
m.setObjective(v,GRB.MAXIMIZE)

In [629]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 35 rows, 24 columns and 71 nonzeros
Model fingerprint: 0x1a098898
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 29 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 14 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7006000e+01   1.400300e+01   0.000000e+00      0s
       6    1.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.100000000e+01


In [630]:
OptimalTime = m.ObjVal
print("MaxFlow:",OptimalTime)

MaxFlow: 11.0


In [631]:
v.x

11.0

### Method2

In [632]:
from gurobipy import *

In [633]:
n = Model()
y = {}
for i in c:
    x1 = {}
    for j in c[i]:
        varname = str('var'+i+j)
        x1[j] = n.addVar(lb = 0, name=varname)
    y[i] = x1

In [634]:
u = {}
for i in c:
    u[i] = n.addVar(lb = 0, name="u"+i)

In [635]:
for i in y:
    for j in y[i]:
        n.addConstr(u[i] - u[j] + y[i][j] >= 0)
        n.addConstr(y[i][j] >= 0)
n.addConstr(-u['S'] + u['T'] == 1)

<gurobi.Constr *Awaiting Model Update*>

In [636]:
for i in y:
    for j in y[i]:
        sum = c[i][j]*y[i][j]
n.setObjective(sum,GRB.MINIMIZE)

In [637]:
n.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 47 rows, 35 columns and 94 nonzeros
Model fingerprint: 0x8d487fef
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 47 rows and 35 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [638]:
n.ObjVal

0.0

In [639]:
u['T']

<gurobi.Var uT (value 1.0)>

In [640]:
u['S']

<gurobi.Var uS (value 0.0)>